# Teil c - Merkmalstandardisierung

# Verbesserung durch Merkmalstandardisierung

## Methode der Standardisierung und Auswirkung auf die Algorithmen

Viele Lernalgorithmen machen es erforderlich die Merkmale zu standardisieren um eine optimale Leistung zu erzielen. Die Algorithmen Perzeptron und Adaline gehören zu den vielen Algorithmen, die von einer Standardisierung profitieren.

Im folgenden wird die Methode <i>Standardisierung</i> erläutert. Diese verleiht den Daten die Eigenschaften einer Standardnormalverteilung. Der Mittelwert jedes Merkmals beträgt 0, die Standardabweichung jeder Spalte beträgt 1. Um zum Beispiel das Merkmal j zu standardiesieren, wird der Mittelwert $\mu$ der jeweiligen Stichprobe abgezogen und das Ergebnis durch die Standardabweichung $\sigma$ dividiert. Das Standardisierungsverfahren wird auf alle Merkmale der Datenmenge angewendet.

$x_j^{\prime(i)} = \frac{x_j^{(i)} - \mu_j}{\sigma_j}$. 

Die Standardisierung verbessert die Algorithmen, weil zum Auffinden einer guten/ optimalen Lösung (das globale Minimum der Straffunktion) weniger Schritte erforderlich sind. Folgende Abbildung stellen die Strafflächen einer zweidimensionalen Klassifizierungsaufgabe als Funktion der Gewichtungen dar.

<img src="./Figures/Merkmalstandardisierung.png" alt="drawing" style="width:500px;"/>


## Implementierung

Selektieren Sie exakt dieselben Daten des Iris-Datensatzes aus Teil a und nehmen Sie die Standardisierung vor. <br>

Trainineren Sie den entweder den Perzeptron-Algorithmus oder den Adaline-Algorithmus aus Teil a mit den standardisierten Daten mit verschiedenen Parametern. <br>

Vergleichen Sie die Ergebnisse zwischen den standardisierten Daten und den nicht-standardisierten Daten. Stellen Sie die verschiedenen Resultate dar.<br>

## Auswahl der Daten

In [1]:
# TODO Implement
X = None
y = None

## Implementierung der Standardisierung

In [2]:
X_std = None # TODO Implement

## Training und Visualisierung des Errors

In [3]:
# TODO: implement